In [2]:
import random

In [3]:
import datasets
hagrid = datasets.load_dataset("miracl/hagrid", split="train")

/home/jiatongy/miniconda3/envs/.venv/lib/python3.12/site-packages/datasets/load.py:1429: FutureWarning: The repository for miracl/hagrid contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/miracl/hagrid
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


**dataset design**  
each instance contains {query, docs, golds}  
query - search query  
docs - total of 10 documents, contain $n \in [1,3]$ positive and $10-n$ negative  
golds - indicate the 3 positives, no ranking for now 

In [2]:
data_len = len(hagrid)

In [3]:
dataset = []

In [4]:
i = 6
len(hagrid[i]['quotes'])

2

In [5]:
hagrid[i]

{'query_id': '3286',
 'query': 'How heavy is an adult golden jackal?',
 'quotes': [{'idx': 0,
   'docid': '26176430#1',
   'text': 'The European jackal is the largest of the golden jackals, with animals of both sexes measuring 120–125\xa0cm (47–49\xa0in) in total length and 10–13\xa0kg (20–29\xa0lb) in body weight. One adult male in North-Eastern Italy is recorded to have reached 14.9\xa0kg (33\xa0lb). The fur is coarse, and is generally brightly coloured with blackish tones on the back. The thighs, upper legs, ears and forehead are bright reddish chestnut. Jackals in Northern Dalmatia have broader than average skulls, which is thought to result from human induced isolation from other populations, thus resulting in a new morphotype.'},
  {'idx': 1,
   'docid': '1554274#13',
   'text': 'The golden jackal is similar to the gray wolf but is distinguished by its smaller size, lighter weight, more elongated torso, less-prominent forehead, shorter legs and tail, and a muzzle that is narrower

In [34]:
relevant_idxs = []
for ans in hagrid[i]['answers']:
    for sent_ist in ans['sentences']:
        if not sent_ist['index'] in relevant_idxs:
            relevant_idxs.append(sent_ist['index'])

In [35]:
relevant_idxs

[0, 1]

In [40]:
hagrid[i]['quotes'][0]['text']

'The European jackal is the largest of the golden jackals, with animals of both sexes measuring 120–125\xa0cm (47–49\xa0in) in total length and 10–13\xa0kg (20–29\xa0lb) in body weight. One adult male in North-Eastern Italy is recorded to have reached 14.9\xa0kg (33\xa0lb). The fur is coarse, and is generally brightly coloured with blackish tones on the back. The thighs, upper legs, ears and forehead are bright reddish chestnut. Jackals in Northern Dalmatia have broader than average skulls, which is thought to result from human induced isolation from other populations, thus resulting in a new morphotype.'

{'query_id': '3280',
 'query': 'When is a language considered dead?',
 'quotes': [{'idx': 0,
   'docid': '1361082#7',
   'text': 'A language is often declared to be dead even before the last native speaker of the language has died. If there are only a few elderly speakers of a language remaining, and they no longer use that language for communication, then the language is effectively dead. A language that has reached such a reduced stage of use is generally considered moribund. Half of the spoken languages of the world are not being taught to new generations of children. Once a language is no longer a native language—that is, if no children are being socialized into it as their primary language—the process of transmission is ended and the language itself will not survive past the current generations.'},
  {'idx': 1,
   'docid': '161708#0',
   'text': 'An endangered language, or moribund language, is a language that is at risk of falling out of use as its speakers die out or shift to sp

TODO: did not limit the number of positive documents yet. need to calculate n.

In [4]:
N = 10 
dataset = []
hagrid_len = len(hagrid)
for inst_idx, inst in enumerate(hagrid):
    query = inst['query']
    # positive instances 
    pos_docs_idxs = []
    for ans in inst['answers']:
        for sent_ist in ans['sentences']:
            if not sent_ist['index'] in pos_docs_idxs:
                pos_docs_idxs.append(sent_ist['index'])
    if len(pos_docs_idxs) > len(inst['quotes']): # this dataset is a bit flawed...
        print("passing an illegal instance...")
        print(inst)
        print()
        continue 

    if len(pos_docs_idxs) > 3: 
        pos_docs_idxs = pos_docs_idxs[:3]
    
    pos_docs = []
    pos_docs_docids = []
    for i in pos_docs_idxs:
        if i >= len(inst['quotes']):
            print('passing an illegal instance...')
            continue
        assert inst['quotes'][i]['docid'] not in pos_docs_docids
        pos_docs_docids.append(inst['quotes'][i]['docid'])
        pos_docs.append(inst['quotes'][i]['text'])

    
    
    neg_docs = []
    neg_docs_docids = []
    while len(neg_docs) < N - len(pos_docs): 
        neg_idx = random.randint(0, hagrid_len-1)
        if neg_idx == inst_idx:
            continue 
        j = random.randint(0, len(hagrid[neg_idx]['quotes'])-1)
        try:
            neg_docid = hagrid[neg_idx]['quotes'][j]['docid']
        except:
            print(neg_idx)
            print(j)
            continue
        if neg_docid not in neg_docs_docids:
            neg_docs_docids.append(neg_docid)
            neg_docs.append(hagrid[neg_idx]['quotes'][j]['text'])
    data_inst = {
        "query": query,
        "docs": pos_docs + neg_docs,
        "golds": pos_docs
    }
    dataset.append(data_inst)

passing an illegal instance...
{'query_id': '3282', 'query': 'Can Orthodox Jewish people eat shellfish?', 'quotes': [{'idx': 0, 'docid': '1769393#11', 'text': 'Tihlo, prepared from roasted barley flour, is very popular in Amhara, Agame, and Awlaelo (Tigray). Traditional Ethiopian cuisine employs no pork or shellfish of any kind, as they are forbidden in the Jewish and Ethiopian Orthodox Christian faiths. It is also very common to eat from the same dish in the center of the table with a group of people.'}, {'idx': 1, 'docid': '9686#4', 'text': 'Both Ethiopian Orthodox Christians and Muslims avoid eating pork or shellfish, for religious reasons. Pork is considered to be unclean in both Ethiopian Orthodox Christianity and Islam. Many Ethiopians would abstain from eating certain meats, and would mostly eat vegetarian and vegan foods.'}, {'idx': 2, 'docid': '36811641#4', 'text': 'Clean Monday is a public holiday in Greece and Cyprus, where it is celebrated with outdoor excursions, the consu

In [10]:
dataset[0]

{'query': 'When is a language considered dead?',
 'docs': ['An endangered language, or moribund language, is a language that is at risk of falling out of use as its speakers die out or shift to speaking another language. Language loss occurs when the language has no more native speakers and becomes a "dead language". If no one can speak the language at all, it becomes an "extinct language". A dead language may still be studied through recordings or writings, but it is still dead or extinct unless there are fluent speakers. Although languages have always become extinct throughout human history, they are currently dying at an accelerated rate because of globalization, neocolonialism and linguicide (language killing).',
  'Linguists distinguish between language "death" and the process where a language becomes a "dead language" through normal language change, a linguistic phenomenon analogous to pseudoextinction. This happens when a language in the course of its normal development graduall

: 

In [59]:
len(dataset)

1327

In [5]:
i = 0
print(dataset[i]['query'])
print()
print(dataset[i]['docs'])
print()
print(dataset[i]['golds'])

When is a language considered dead?

['An endangered language, or moribund language, is a language that is at risk of falling out of use as its speakers die out or shift to speaking another language. Language loss occurs when the language has no more native speakers and becomes a "dead language". If no one can speak the language at all, it becomes an "extinct language". A dead language may still be studied through recordings or writings, but it is still dead or extinct unless there are fluent speakers. Although languages have always become extinct throughout human history, they are currently dying at an accelerated rate because of globalization, neocolonialism and linguicide (language killing).', 'Linguists distinguish between language "death" and the process where a language becomes a "dead language" through normal language change, a linguistic phenomenon analogous to pseudoextinction. This happens when a language in the course of its normal development gradually morphs into something

: 